In [1]:
!pip install shapely geopandas
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from shapely import wkb
from geopandas import GeoDataFrame
from shapely import wkb, Point, LineString
from pyspark.sql import Row
import pyspark.pandas as ps
import math

ps.set_option('compute.default_index_type', 'distributed')
spark = SparkSession.builder.appName("pyspark-notebook").master("spark://spark-master:7077").config("spark.executor.memory", "512m").getOrCreate()

#dataframe = spark.read.format("jdbc").options(url="jdbc:postgresql://db:5432/datascience?user=postgres&password=mantequilla", dbtable="rutas",driver="org.postgresql.Driver").load()
buses = spark.read.option("header",True).csv("./data/buses.csv")
paraderos2 = spark.read.option("header",True).csv("./data/paraderos2.csv")
paraderosrutas = spark.read.option("header",True).csv("./data/paraderosrutas.csv")
print("Hola")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/18 14:18:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Hola


In [2]:
print("Hey")
#jdbcDF = spark.read
#  .format("jdbc")
#  .option("url", "jdbc:postgresql:dbserver")
#  .option("dbtable", "schema.tablename")
#  .option("user", "username")
#  .option("password", "password")
#  .load()

def distance(p0, p1):
    return math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2)
def length(s) -> int:
    return len(s)
def some(item):
    punto_bus = wkb.loads(item["geom_bus"], hex=True)
    punto_paradero = wkb.loads(item["parder_geom"], hex=True)
    distancia = punto_bus.distance(punto_paradero)
    #x,y = punto.xy
    #px, py = list(x)[0], list(y)[0]
    temp = item
    temp["distancia"] = distancia
    return temp  #punto #(px, py)

def createData(buses, paraderos2, paraderosrutas):
    # Observacion: Hay que cambiar la obtencion de parder_geom, puesto que debiese venir de la tabla paraderos2 pero el csv de stops.csv venía sin ese campo
    busesrutas = paraderosrutas.join(buses, buses.vendor == paraderosrutas.cod_sinser).select(paraderosrutas.simt, 
        buses.bus, buses.fechahora, buses.lat, buses.lng, buses.velocidad, buses.pasajeros, buses.vendor, buses.id, buses.geom.alias("geom_bus"),
        paraderosrutas.servicio, paraderosrutas.nombre_par, paraderosrutas.geom.alias("parder_geom"))
    
    """
    data = busesrutas.join(paraderos2, paraderos2.stop_id == busesrutas.simt, 'inner').select(busesrutas.columns + [paraderos2.stop_id.alias('parader'), 
        paraderos2.stop_name, paraderos2.stop_lat.alias('parder_lat'), paraderos2.stop_lon.alias('parder_lon'), busesrutas.geom_bus
    ])
    """
    data = busesrutas.join(paraderos2, paraderos2.stop_id == busesrutas.simt, 'inner').select(busesrutas.simt, busesrutas.bus, busesrutas.fechahora, busesrutas.lat, busesrutas.lng, busesrutas.velocidad, busesrutas.pasajeros, busesrutas.vendor, busesrutas.id, busesrutas.geom_bus, busesrutas.servicio, busesrutas.nombre_par, busesrutas.parder_geom, paraderos2.stop_id.alias('parader'), paraderos2.stop_name, paraderos2.stop_lat.alias('parder_lat'), paraderos2.stop_lon.alias('parder_lon'))
    #data.printSchema()
    geom = data.select(data.columns).dropna().coalesce(3)

    #data.foreachPartition(some)
    data_pd = geom.to_pandas_on_spark()#.coalesce(3)
    data_pd_chunk = data_pd.spark.coalesce(3)

    new_geom = data_pd_chunk.apply(some, axis='columns')
    # axis = 1 --> foreach row : len(row) : numbers of columns
    # axis = 1 --> foreach column : len(column) : numbers of rows by columns
    print("After")
    # Next 3 lines are only when apply function is used
    #extra = new_geom.to_frame()
    #print(type(extra))
    #extra.head(3)

    #new_geom.head(3)
    #print(type(new_geom))
    busesparaderos = new_geom.to_spark()
    #busesparaderos.printSchema()
    temp_minimos = busesparaderos.groupBy('fechahora').agg(F.min(busesparaderos.distancia).alias("md"))
    temp_minimos.printSchema()
    minimos = temp_minimos.select(temp_minimos.md,temp_minimos.fechahora.alias("fh"))

    resultados = busesparaderos.join(minimos, [busesparaderos.distancia == minimos.md , busesparaderos.fechahora == minimos.fh], 'inner')
    return resultados


Hey


In [3]:


word = '0101000020E61000000200008017AD51C0010000C01CBC40C0'
punto = wkb.loads(word, hex=True)
x,y = punto.xy
word = Row(name="Alice")
arr = Point()
print(len(word), word, Row(word.asDict().items()))


1 Row(name='Alice') <Row(dict_items([('name', 'Alice')]))>


In [4]:
resultados = createData(buses,paraderos2, paraderosrutas)

After
root
 |-- fechahora: string (nullable = true)
 |-- md: double (nullable = true)



In [ ]:
resultados.printSchema()
temp2 = resultados.to_pandas_on_spark().spark.coalesce(3).groupby(["parader"]).apply(lambda x: x)

root
 |-- simt: string (nullable = true)
 |-- bus: string (nullable = true)
 |-- fechahora: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)
 |-- velocidad: string (nullable = true)
 |-- pasajeros: string (nullable = true)
 |-- vendor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- geom_bus: string (nullable = true)
 |-- servicio: string (nullable = true)
 |-- nombre_par: string (nullable = true)
 |-- parder_geom: string (nullable = true)
 |-- parader: string (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- parder_lat: string (nullable = true)
 |-- parder_lon: string (nullable = true)
 |-- distancia: double (nullable = true)
 |-- md: double (nullable = true)
 |-- fh: string (nullable = true)



In [6]:

filtrados = resultados.filter(resultados.vendor == 'E04I')


In [7]:
print(type(filtrados), type(resultados))

<class 'pyspark.sql.dataframe.DataFrame'> <class 'pyspark.sql.dataframe.DataFrame'>


In [8]:
#filtrados.head(3)
#filtrados.show(2)
muestras = filtrados.to_pandas_on_spark()
print("End")
new_muestras = muestras.spark.coalesce(3)

End


In [9]:
#muestras.collect()
# new_muestras.head(3)

In [10]:
# 

In [11]:

def get_Bus_Cant_Prom_Por_Paredero(source):#Frecuenica
    Hour_list_from = ['2018-06-13 00:00:00+00','2018-06-13 01:00:00+00','2018-06-13 02:00:00+00','2018-06-13 03:00:00+00','2018-06-13 04:00:00+00','2018-06-13 05:00:00+00','2018-06-13 06:00:00+00','2018-06-13 07:00:00+00','2018-06-13 08:00:00+00','2018-06-13 09:00:00+00','2018-06-13 10:00:00+00','2018-06-13 11:00:00+00','2018-06-13 12:00:00+00','2018-06-13 13:00:00+00','2018-06-13 14:00:00+00','2018-06-13 15:00:00+00','2018-06-13 16:00:00+00','2018-06-13 18:00:00+00','2018-06-13 19:00:00+00','2018-06-13 20:00:00+00','2018-06-13 21:00:00+00','2018-06-13 22:00:00+00','2018-06-13 23:00:00+00']
    Hour_list_to = ['2018-06-13 00:59:59+00','2018-06-13 01:59:59+00','2018-06-13 02:59:59+00','2018-06-13 03:59:59+00','2018-06-13 04:59:59+00','2018-06-13 05:59:59+00','2018-06-13 06:59:59+00','2018-06-13 07:59:59+00','2018-06-13 08:59:59+00','2018-06-13 09:59:59+00','2018-06-13 10:59:59+00','2018-06-13 11:59:59+00','2018-06-13 12:59:59+00','2018-06-13 13:59:59+00','2018-06-13 14:59:59+00','2018-06-13 15:59:59+00','2018-06-13 16:59:59+00','2018-06-13 18:59:59+00','2018-06-13 19:59:59+00','2018-06-13 20:59:59+00','2018-06-13 21:59:59+00','2018-06-13 22:59:59+00','2018-06-13 23:59:59+00']
    dataframe_ruta = source#.toPandas()
    
    #temp2 = dataframe_ruta.groupby(['parader'], group_keys=True, as_index=True).apply(lambda x: x)
    temp2 = dataframe_ruta.groupBy(['parader']).apply(lambda x: x)
    return 
    #temp4 = dataframe_ruta.groupby(['parder_geom'], group_keys=True, as_index=True).apply(lambda x: x)
    list_paredors = temp2.index.levels[0]
    #temp3 = temp2.loc['PA146']
    #a = temp2.loc['PA146']['fechahora'].tolist()[0]

    dataframe_buses = {'paradero':[],'promedio_bus_hora':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[]}
    #print(len(Hour_list_from))
    #print(len(Hour_list_to))

    for i in list_paredors: 
        x = temp2.loc[i]
        List_total_buses = list()
        geo = x['parder_geom'].iat[0]
        geo_lat = x['parder_lat'].iat[0]
        geo_lon = x['parder_lon'].iat[0]
        #paradero_list = list()
        #print(geo)
        for j in range(23):
            #print(j)
            df = x[(x['fechahora'] >= Hour_list_from[j]) & (x['fechahora'] <= Hour_list_to[j])]
            List_total_buses.append(df['bus'].nunique())
            #Lista_interval.append(Hour_list_from[j]+' to '+Hour_list_to[j])
        mean = (sum(List_total_buses) / 24)
        dataframe_buses['paradero'].append(i)
        dataframe_buses['promedio_bus_hora'].append(mean)
        dataframe_buses['geo_parader'].append(geo)
        dataframe_buses['geo_parader_lat'].append(geo_lat)
        dataframe_buses['geo_parader_lon'].append(geo_lon)
        if mean < 5:
            dataframe_buses['calificacion'].append(0)
        elif mean > 6:
            dataframe_buses['calificacion'].append(1)
        else:
            dataframe_buses['calificacion'].append(2)


        
        #dataframe_buses['rango_hora'] += Lista_interval


    dataframe_total_busparadero = pd.DataFrame(dataframe_buses)

#print(resultados.to_pandas_on_spark().shape)
#get_Bus_Cant_Prom_Por_Paredero(resultados)

In [12]:
# delete.item()
print("Get item from a Series")
!pip install koalas

Get item from a Series


In [13]:
import databricks.koalas as ks

ks.set_option('compute.default_index_type', 'distributed')
#data.count()
#kf = ks.DataFrame(data)
#len(kf)
#kf.apply(length, 1)
#df2 = spark.sql("select * from mytable")
#df2.count()


kdf = ks.DataFrame({'a': [1,2,3], 'b':[4,5,6]})
def pandas_plus(pser):
    suma = sum(pser)
    temp = pser
    temp["suma"] = suma
    return temp  # allows an arbitrary length
#kdf.print_schema()
kdf.apply(pandas_plus, axis='columns')

#example = ps.DataFrame(resultados)
#other = example.groupby(['parader']).apply(lambda x: x)


,a,b,suma
0,1,4,5
1,2,5,7
2,3,6,9
